In [ ]:
from matplotlib import pyplot as plt
import torch

# sample random percolation system
def sample_percolation(L,p,random_state=0):
    g=None
    if random_state is not None:
        g = torch.Generator().manual_seed(random_state)
        
    system = (torch.rand((L,L),generator=g)<p)*1
    return system

# size of system
L = 20
# concentration
p=[0.1,0.2,0.5,0.8]

plt.figure(figsize=(len(p)*4,4))
for i,p0 in enumerate(p):
    
    system=sample_percolation(L,p0,0)
    plt.subplot(1,len(p),1+i)
    plt.imshow(system)
    plt.axis('off')
    plt.title(f"p={p0}; total={system.sum()}")

In [ ]:
# this is test of uniformity of out percolation system distribution
from scipy.stats import chi2

def test_for_uniformity(system,p):

    # convert matrix to point's coordinates
    XY = torch.stack(torch.where(system==1),-1)*1.0

    # use Scott's formula
    # compute count of categories k for each dimension for pearson's test
    xy_std = XY.std(0)
    h = 3.5*xy_std/len(XY)**(1/3/2) # we half power for len(XY) because we compute 2d grid
    R = XY.max(0)[0]-XY.min(0)[0] # radius of our data
    k=(R/h) # scott's formula

    # round k to integer
    k=k.round().long()

    # peek first k that divides system into chunks
    while size%k[0]!=0:
        k[0]+=1
    
    while size%k[1]!=0:
        k[1]+=1

    # chunk system
    chunks = system.reshape(*k,-1)

    # compute 2d histogram
    hist2d = chunks.sum(-1)

    # expected count of element in a cell is just size of cell multiplied with probability p
    expected = chunks.shape[-1]*p

    # compute chi2 value
    chi2_value = ((hist2d-expected).pow(2)/expected).sum().item()


    # we have k[0]*k[1] unknowns in each of cell, so each of them defines one degree of freedom
    # meanwhile we know what p is, so one degree of freedom is removed
    # because of these our tests are not quite independent, so we add degrees of freedom to out pearson test
    df = k[0]*k[1]-1
    p_value = chi2.sf(chi2_value, df)
    
    return hist2d, chi2_value, p_value

size = 1000
p=0.5
system = sample_percolation(size,p,None)
hist2d, chi2_value, p_value = test_for_uniformity(system,0.5)

print("p is",p)
print("Computed chi2 test",chi2_value)
print("P-value is",p_value)

plt.imshow(system)
plt.axis('off')
plt.title("Percolation system")
plt.show()


plt.imshow(hist2d)
plt.colorbar(label="Value")
plt.title("Binned percolation system")
plt.axis('off')
plt.show()


In [ ]:
# seed for random number generator
random_state = None

# size of grid
L=1000

# count of experiment repeats
repeats = 20

results_mean = []
results_std = []
probabilities = [p/10 for p in range(1,10)]

for p in probabilities:
    p_values = []
    for i in range(repeats):
        system = sample_percolation(L,p,random_state)
        p_value = test_for_uniformity(system,p)[-1]
        p_values.append(p_value)
    p_values=torch.tensor(p_values)
    results_mean.append(p_values.mean().item())
    results_std.append(p_values.std().item())

In [ ]:
plt.figure(figsize=(6,4))
plt.errorbar(probabilities, results_mean, yerr=results_std, fmt='o-', capsize=5, elinewidth=1.5, markersize=6, color="navy")

plt.xlabel("Percolation probability")
plt.ylabel("P-value for uniformity pearson test")
plt.title(f"Mean ± Std over {repeats} experiments")
plt.grid(True, linestyle="--", alpha=0.6)
plt.show()